In [1]:
import os
from dotenv import load_dotenv
import random
import torch
import torch.nn.functional as F
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
import pandas as pd
load_dotenv()

hugkey = os.getenv('HUGGINGFACE_KEY_LLAMA')
login(hugkey)

In [2]:
model_name = "meta-llama/Llama-3.2-3B-Instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_name, device_map='auto')
# tokenizer.pad_token is None

In [3]:
# model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto', load_in_8bit=True)
# model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_name, device_map='auto')
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

This is my current draft (see interactive_chat.py)

---

In [6]:
from transformers.generation.streamers import BaseStreamer
import time

class CallbackStreamer(BaseStreamer):
    def __init__(self, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer

    def put(self, value):
        decoded = self.tokenizer.batch_decode(value, skip_special_tokens=True)
        import pdb; pdb.set_trace()
        print(decoded[0], end="")
        # time.sleep(0.01)

    def end(self):
        print()

streamer = CallbackStreamer(tokenizer)

In [7]:
# prompt = ["What is the capital of France?"]
chat = [
    {'role': 'system', 'content': "You're a helpful chatbot that answers questions and solves the tasks you're given. You're direct and only expand on a topic if this is requested by the user. You only provide correct information and say it when you don't know the answer."},
    {'role': 'user', 'content': "Hi! Can you tell me what is the capital of France? Also tell me about how world war 2 started."},
]
chat_text = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
# print(chat_text)
input_ids = tokenizer(chat_text, padding=True, return_tensors="pt").to('cuda')
outputs = model.generate(**input_ids, do_sample=True, max_length=250, streamer=streamer)
tokenizer.decode(outputs[0][input_ids['input_ids'].shape[-1]:], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


> c:\users\sofia\appdata\local\temp\ipykernel_21256\2497998121.py(12)put()

tensor([[128000, 128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,
           2696,     25,   6790,    220,   2366,     18,    198,  15724,   2696,
             25,    220,   2705,  10263,    220,   2366,     20,    271,   2675,
           2351,    264,  11190,   6369,   6465,    430,  11503,   4860,    323,
          68577,    279,   9256,    499,   2351,   2728,     13,   1472,   2351,
           2167,    323,   1193,   9407,    389,    264,   8712,    422,    420,
            374,  11472,    555,    279,   1217,     13,   1472,   1193,   3493,
           4495,   2038,    323,   2019,    433,    994,    499,   1541,    956,
           1440,    279,   4320,     13, 128009, 128006,    882, 128007,    271,
          13347,      0,   3053,    499,   3371,    757,   1148,    374,    279,
           6864,    315,   9822,     30,   7429,   3371,    757,    922,   1268,
           1917,   4208,    220, 

In [1]:
import os
import time
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers.generation.streamers import BaseStreamer
from huggingface_hub import login


class CallbackStreamer(BaseStreamer):
    def __init__(self, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer

    def put(self, value):
        decoded = self.tokenizer.batch_decode(value, skip_special_tokens=True)[0]
        breakpoint()
        if len(decoded) > 50:
            return
        print(decoded, end="")
        time.sleep(0.1)

    def end(self):
        print("\n")


def main(model_name, quantize=True):
    """
    Support an interactive chat with the model in the terminal.
    """
    bnb_config = BitsAndBytesConfig(load_in_8bit=quantize)
    model = AutoModelForCausalLM.from_pretrained(
        model_name, device_map='auto', quantization_config=bnb_config, 
    )
    model.eval()
    tokenizer = AutoTokenizer.from_pretrained(model_name, device_map='auto')
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id
        model.generation_config.pad_token_id = tokenizer.eos_token_id
    streamer = CallbackStreamer(tokenizer)

    chat = [
        {'role': 'system', 'content': "You're a helpful chatbot that answers questions and solves the tasks you're given. You're direct and only expand on a topic if this is requested by the user. You only provide correct information and say it when you don't know the answer."},
        {'role': 'assistant', 'content': "Hi Sofia! How can I help you today?"},
    ]
    print(f"\nYou're chatting with {model_name}. Press q to exit.")
    print("\n\033[0;32m<llm>\033[0;0m " + chat[-1]['content'] + "\n")

    while True:
        user_input = input("\033[0;32m<user>\033[0;0m ")
        if user_input == "q":
            break
        chat.append({'role': 'user', 'content': user_input})
        chat_text = tokenizer.apply_chat_template(
            chat, tokenize=False, add_generation_prompt=True,
        )
        print("\n\033[0;32m<llm>\033[0;0m ", end="")
        inputs = tokenizer(chat_text, padding=True, return_tensors='pt').to('cuda')
        output = model.generate(
            **inputs, streamer=streamer,
            do_sample=True,
            max_new_tokens=512,
        )
        response = tokenizer.decode(
            output[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True,
        )[0]
        chat.append({'role': 'assistant', 'content': response})


from dotenv import load_dotenv
load_dotenv()
hugkey = os.getenv('HUGGINGFACE_KEY_LLAMA')
login(hugkey)

model_name = "meta-llama/LLama-3.2-3B-Instruct"
main(model_name, quantize=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


You're chatting with meta-llama/LLama-3.2-3B-Instruct. Press q to exit.

<llm> Hi Sofia! How can I help you today?


<llm> What subject and what's the specific question or task you're struggling with? I'll do my best to assist you.



Here is my first draft of the code

---

In [ ]:
def get_probability(logits, token, tokenizer):
    """Calculate the probability of the token given the logits"""
    assert token in tokenizer.vocab, "token not in tokenizer vocabulary"
    probs = F.softmax(logits.logits, dim=-1)  # (B, length, vocab)
    return probs[:, -1, tokenizer.vocab[token]]


def llm_forward(model, tokens):
    """
    A forward pass through the model.

    Args:
        model: Huggingface model.
        tokens (dic): contains input_ids and attention_mask of (num_sen, len_sen)
    
    Returns:
        probs (tensor): a (num_sen, vocab) probability distribution for next token
    """
    # Preliminares: idx for sentences and for padding start
    num_sen, _ = tokens['input_ids'].shape
    i = torch.arange(0, num_sen, device='cuda')               # (num_sen, )
    mask_limit = torch.sum(tokens['attention_mask'], dim=-1)  # (num_sen, )

    # Run the model and extract probabilities for next token
    with torch.no_grad():
        logits = model(**tokens).logits          # (num_sen, len_sen, vocab)
        probs = F.softmax(logits[i, mask_limit - 1], dim=-1)  # (num_sen, vocab)
    return probs


def autoregressive_forward_topk(model, tokenizer, sentences, k=1):
    """
    Does a single pass of the model and outputs the top k predictions.
    Organizes the ids, tokens, and probabilities in a dataframe
    """
    # Calculate top predictions
    tokens = tokenizer(sentences, padding=True, return_tensors='pt').to('cuda')
    probs = llm_forward(model, tokens)
    topk_probs, topk_ids = torch.topk(probs, k, dim=-1)  # (num_sen, k)
    topk_tokens = [tokenizer.batch_decode(sentence_toks) for sentence_toks in topk_ids]

    # Construct dataframe
    data = []
    num_sentences = topk_ids.shape[0]
    for sent_idx in range(num_sentences):
        for guess_idx in range(k):
            data.append({
                'sentence': str(sent_idx) + ": " + sentences[sent_idx],
                'num_guess': guess_idx,
                'token': topk_tokens[sent_idx][guess_idx],
                'token_id': topk_ids[sent_idx][guess_idx].item(),
                'probability': topk_probs[sent_idx][guess_idx].item()
            })
    df = pd.DataFrame(data).set_index(['sentence', 'num_guess'])
    df = df.sort_index(axis=0, ascending=[True, True])
    return df


def llm_single_pass(model, tokens, k=1):
    """
    Does a single pass of the model and chooses randomly from
        the top k options to continue the text.

    Args:
        model: HuggingFace model
        tokens (token item): input_ids has shape (num_sen, len_sen)
        k (int, optional): number of hypothesis to consider

    Returns:
        pred_tokens: tensor of shape (num_sen, ) with tokens to add
    """
    # Forward pass
    probs = llm_forward(model, tokens)
    topk_probs, topk_tokens = torch.topk(probs, k, dim=-1)  # (num_sen, k)

    # Choose element at random
    pred_tokens = torch.empty((probs.shape[0], ), dtype=torch.int64, device='cuda')
    for i, (sen_prob, sen_tokens) in enumerate(zip(topk_probs, topk_tokens)):
        pred_id = random.choices(range(k), weights=sen_prob)
        pred_tokens[i] = sen_tokens[pred_id]
    return pred_tokens


def llm_pipeline(model, tokenizer, sentences, k=1, max_tokens=32):
    """
    Iteratively complete each sentence using the specified model.
        Each step randomly chooses one of the top k options.

    Args:
        model, HuggingFace model
        tokenizer, HuggingFace tokenizer
        sentences: List[str], list of sentences to give the model
        k: Optional[int], number of hypothesis to consider
        max_tokens: Optional[int], number of tokens to add to each sentence

    Returns:
        sentences: list of completed sentences
    """
    # Create tokens of shape (num_sen, len_sen)
    tokens = tokenizer(sentences, padding=True, return_tensors='pt').to('cuda')

    # Define idx for sentences and idx where the masking starts
    i = torch.arange(0, len(sentences), device='cuda')        # (num_sen, )
    mask_limit = torch.sum(tokens['attention_mask'], dim=-1)  # (num_sen, )
    extra_padding = torch.full(
        size=(len(sentences), 1),
        fill_value=tokenizer.vocab[tokenizer.special_tokens_map['pad_token']],
        device='cuda',
    )

    # Main loop: use model repeteadly and iterate over tokens
    for _ in range(max_tokens):
        # NOTE: Simplification: all mask limits are incremented in each step
        new_tokens = llm_single_pass(model, tokens, k)            # (num_sen, )
        token_ids = torch.cat((tokens['input_ids'], extra_padding.clone()), dim=-1)
        attn_mask = torch.cat((tokens['attention_mask'], torch.zeros_like(extra_padding)), dim=-1)

        token_ids[i, mask_limit] = new_tokens
        attn_mask[i, mask_limit] = 1
        mask_limit += 1

        tokens = {'input_ids': token_ids, 'attention_mask': attn_mask}

    # Convert back to sentences
    sentences = [
        tokenizer.decode(t, skip_special_tokens=True) for t in tokens['input_ids']
    ]
    return sentences

In [5]:
chats = [
    "Most people wouldn't bat an eye if",
    "Pasta is a popular dish in many",
    "El salvador is well known because",
]
df = autoregressive_forward_topk(model, tokenizer, chats, k=5)
df

token  token_id  \
sentence                              num_guess                          
0: Most people wouldn't bat an eye if 0                 they       814   
                                      1                  you       499   
                                      2                    a       264   
                                      3                their       872   
                                      4              someone      4423   
1: Pasta is a popular dish in many    0             cultures     27833   
                                      1            countries      5961   
                                      2                parts      5596   
                                      3              Italian     15155   
                                      4           households     29939   
2: El salvador is well known because  0                   of       315   
                                      1                   it       433   
                                      2                  the       279   
                                      3                  its      1202   
                                      4                 they       814   

                                                 probability  
sentence                              num_guess               
0: Most people wouldn't bat an eye if 0             0.389648  
                                      1             0.247681  
                                      2             0.076721  
                                      3             0.072083  
                                      4             0.051910  
1: Pasta is a popular dish in many    0             0.317139  
                                      1             0.195435  
                                      2             0.177856  
                                      3             0.060516  
                                      4             0.054260  
2: El salvador is well known because  0             0.787109  
                                      1             0.095459  
                                      2             0.023041  
                                      3             0.020020  
                                      4             0.007656

In [6]:
tokens = tokenizer(chats, padding=True, return_tensors='pt').to('cuda')
llm_forward(model, tokens)

tensor([[2.3842e-07, 2.3842e-07, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.7881e-07, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [3.3975e-06, 8.3447e-07, 5.9605e-08,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]], device='cuda:0', dtype=torch.float16)

In [ ]:
outputs = llm_pipeline(model, tokenizer, chats, k=10, max_tokens=64)
for sen in outputs:
    print(sen + "\n\n")

Most people wouldn't bat an eye if a stranger asked if they could borrow their phone to call for an ambulance. But when that stranger is a police officer, the answer may come with some hesitation. That's because a new bill introduced in the House of Commons on Thursday aims to give police officers the power to seize your phone in an emergency.
The proposal,



Pasta is a popular dish in many households, both for lunch and for dinner. However, the preparation of pasta often requires the use of a lot of cooking oil. This can lead to an increase in your daily calorie consumption, which can be counter-productive if you are trying to lose weight. Fortunately, there is a way to reduce pasta’s calorie count



El salvador is well known because of the coffee. It is a popular commodity for the coffee lovers worldwide. It is a country which produces and exports coffee in great volumes. It is a well known coffee bean and it is very famous in the whole world.
El Salvador is well known as a coffee 

Terminal output

---

The idea is to format the python text after it's been written, and override the plain text there was before.

Or maybe not showing the code as plain text and wait for it to finish.

Indentation problems don't happen in the terminal.

Render it each time a new \n is written.

In [32]:
from rich.console import Console, Group
from rich.syntax import Syntax

console = Console()

code = """def foo(x):\n\tprint(x)"""
# code = """a = \sum_{n=0}^\infty x^n"""
syntax = Syntax(code, "python", theme='monokai', line_numbers=False)
group = Group("Here's a code snippet:\n", syntax)

print("Python functions are defined with the def keyword.")
console.print(group)

Python functions are defined with the def keyword.


Here's a code snippet:

def foo(x):                                                                                                        
    print(x)                                                                                                       

In [10]:
for i in range(128000):
    token = tokenizer.decode([i])
    # if "\n" in token and token[-1] != "\n":
        # print("#" + token + "*")
    # if "\t" in token:
    if "```" in token:
        print(i)

42333
55375
74694


In [39]:
max((len(tok), i) for tok, i in tokenizer.vocab.items())

(128, 58040)

In [7]:
tokenizer.decode([42333])

' ```\n'